In [1]:
!pip install torch torchvision numpy gradio
!pip install pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 26.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [2]:
!pip install transformers[torch] --upgrade
!pip install accelerate --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [3]:
!pip install huggingface-hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [4]:
import pandas as pd

# Load the CSV file to check the column name
df = pd.read_csv('/content/derogatory_corpus.csv')
print("Column names in the CSV file:", df.columns.tolist())


Column names in the CSV file: ['Fuck niggers in general are the worst, but this one in particular is the fucking worst of the worst.', 'Unnamed: 1']


In [6]:
import gradio as gr
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import pandas as pd
import torch
import os


os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Define a function to set up the GPT-Neo model and tokenizer
def setup_model():
    model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
    tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3.B")
    model.to("cuda")
    return model, tokenizer

model, tokenizer = setup_model()

def fine_tune_model(model, tokenizer, file_path):
    df = pd.read_csv(file_path)
    text = df['Fuck niggers in general are the worst, but this one in particular is the fucking worst of the worst.'].tolist()

    with open('temp_text.txt', 'w') as f:
        for item in text:
            f.write("%s\n" % item)

    dataset = TextDataset(tokenizer=tokenizer, file_path='temp_text.txt', block_size=128)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir="./gptneo_finetuned",
        overwrite_output_dir=True,
        num_train_epochs=5,
        per_device_train_batch_size=2,  # batch size
        gradient_accumulation_steps=16,
        save_steps=10_000,
        save_total_limit=2,
        prediction_loss_only=True,
        logging_dir='./logs',
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    trainer.train()
    trainer.save_model("./gptneo_finetuned")
    return model

torch.cuda.empty_cache()

fine_tuned_model = fine_tune_model(model, tokenizer, '/content/derogatory_corpus.csv')

def create_adversarial_image(image, clip_model, clip_processor, target_desc="This scene is deceptive"):
    """
    Modify an image to subtly cue the GPT-Neo model to generate text that aligns with a harmful or deceptive target description.
    """
    # Process the original image through CLIP to get baseline features
    inputs = clip_processor(images=image, return_tensors="pt", padding=True)
    original_features = clip_model.get_image_features(**inputs)

    # Define the target text and process it through CLIP to get the desired text features
    text_inputs = clip_processor(text=[target_desc], return_tensors="pt", padding=True)
    target_features = clip_model.get_text_features(**text_inputs)

    # Calculate the loss to maximize the cosine similarity between modified image features and target text features
    loss = -torch.nn.functional.cosine_similarity(original_features, target_features).mean()
    loss.backward()

    # Apply calculated gradients to the original image to create an adversarial version
    step_size = 0.01  # Small step size for subtle modification
    adversarial_image = inputs.pixel_values - step_size * inputs.pixel_values.grad
    adversarial_image = adversarial_image.detach()  # Detach from the current graph
    return adversarial_image.squeeze()  # Remove unnecessary dimensions

def generate_text_from_adversarial_image(adversarial_image, prompt, model, tokenizer, max_length=100, temperature=0.8):
    """
    Generate text from a model using an adversarial image and a given text prompt.
    """
    # Decode the adversarial image features to text description (hypothetical)
    adversarial_description = clip_processor.decode(adversarial_image, skip_special_tokens=True)
    combined_prompt = f"{adversarial_description} {prompt}"
    inputs = tokenizer(combined_prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs['input_ids'],
        max_length=max_length,
        temperature=temperature,
        num_beams=5,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


title = "Text Generation Demo Using Fine-Tuned GPT-Neo"
description = "This demo uses a fine-tuned GPT-Neo model on a specific adversarial corpus to generate text based on your prompts."

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Optional Image Upload (Not Used)")
            prompt_input = gr.Textbox(label="Enter Your Prompt", placeholder="Type your prompt here...", lines=4)
            max_length = gr.Slider(minimum=20, maximum=100, value=50, step=1, label="Max Length for Generated Text")
            temperature = gr.Slider(minimum=0.5, maximum=1.0, value=0.8, step=0.05, label="Temperature")
            generate_button = gr.Button("Generate Text")
        with gr.Column():
            output_text = gr.Textbox(label="Generated Text", placeholder="The generated text will appear here...", lines=12)

    generate_button.click(fn=generate_text, inputs=[prompt_input, max_length, temperature], outputs=output_text)

demo.launch()


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7ee75fa6f5ff01cf7b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
